# Data Analysis

1. [Get cleaned data](#BGet-cleaned-data)
2. [Get Zone Name and attach to data](#Get-Zone-Name-and-attach-to-data)
3. [Histogram Comparison: Pickups per Zone](#Histogram-Comparison,-Pickups-per-Zone)
4. [Linear Chart: Pickups evolution over time](#Linear-Chart,-Pickups-evolution-over-time)
5. [Scatter Plot, Relation between Precipitation and Pickups](#Scatter-Plot,-Relation-between-Precipitation-and-Pickups)
6. [Pairwise Relationships](#Pairwise-Relationships)
7. [Pickups in Rainy Day vs Not Rainy Days](#Pickups-in-Rainy-Day-vs-Not-Rainy-Days)
8. [Some conclusions](#Some-conclusions)

In [61]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import matplotlib.pyplot as plt
plt.style.use("seaborn")
plt.rcParams["figure.figsize"] = (15, 5)
import seaborn as sns

pd.options.display.max_columns = None
pd.options.display.max_rows = None
import altair as alt

# alt.data_transformers.disable_max_rows()

In [30]:
%matplotlib inline

# Get cleaned data

In [31]:
year=2017

df = pd.read_csv('../data/Data_Cleaned_'+str(year)+'_To_Model.csv', sep=',',
                       parse_dates=['datetime'])
df = df.rename(columns={'NoOfPickups':'pickups'})

df.head()

,datetime,month,day,hour,LocationID,pickups,year,week,dayofweek,isweekend,precipitation
0,2017-01-01,1,1,0,4,136.0,2017,52,6,1,0.0
1,2017-01-01,1,1,0,12,3.0,2017,52,6,1,0.0
2,2017-01-01,1,1,0,13,103.0,2017,52,6,1,0.0
3,2017-01-01,1,1,0,24,94.0,2017,52,6,1,0.0
4,2017-01-01,1,1,0,41,136.0,2017,52,6,1,0.0


# Get Zone Name and attach to data

In [32]:
# 1. Import Location and Borough columns form NY TAXI ZONES dataset
dfzones = pd.read_csv('../data/NY_taxi_zones.csv', sep=',',
                      usecols=['LocationID', 'borough', 'zone'])

# 2. Filter Manhattan zones
dfzones = dfzones[dfzones['borough']=='Manhattan']\
                .drop(['borough'], axis=1)\
                .sort_values(by='LocationID')\
                .drop_duplicates('LocationID').reset_index(drop=True)
dfzones.head()

,zone,LocationID
0,Alphabet City,4
1,Battery Park,12
2,Battery Park City,13
3,Bloomingdale,24
4,Central Harlem,41


In [33]:
df2 = df.merge(dfzones, left_on='LocationID', right_on='LocationID')
df2.head()

,datetime,month,day,hour,LocationID,pickups,year,week,dayofweek,isweekend,precipitation,zone
0,2017-01-01 00:00:00,1,1,0,4,136.0,2017,52,6,1,0.0,Alphabet City
1,2017-01-01 01:00:00,1,1,1,4,144.0,2017,52,6,1,0.0,Alphabet City
2,2017-01-01 02:00:00,1,1,2,4,189.0,2017,52,6,1,0.0,Alphabet City
3,2017-01-01 03:00:00,1,1,3,4,186.0,2017,52,6,1,0.0,Alphabet City
4,2017-01-01 04:00:00,1,1,4,4,125.0,2017,52,6,1,0.0,Alphabet City


# Histogram Comparison, Pickups per Zone
The plot shows Manhattan zones ordered by their average number of pickups over the year.

The top ten zones by pickups here:

In [34]:
df_grouped = df2[['zone', 'pickups']]
df_grouped = df_grouped.groupby('zone').mean().sort_values('pickups', ascending=False).reset_index()
display(df_grouped.head(10))

# all zones sum
all_sum = df_grouped.sum(axis=0)[1]

# top 10 zones sum
df_grouped_top10 = df_grouped[0:10]
top10_sum = df_grouped_top10.sum(axis=0)[1]

print('Total sum of pickups: ', int(all_sum))
print('Top 10 zones sum of pickups: ', int(top10_sum))
print('The top 10 zones get {0}% of the pickups'.format(int(top10_sum/all_sum*100)))

,zone,pickups
0,Upper East Side South,505.161416
1,Midtown Center,485.399543
2,Upper East Side North,457.652283
3,Penn Station/Madison Sq West,451.312671
4,Midtown East,438.765411
5,Times Sq/Theatre District,435.514155
6,Union Sq,420.867009
7,Murray Hill,412.360616
8,Clinton East,404.736301
9,East Village,377.901941


Total sum of pickups:  11705
Top 10 zones sum of pickups:  4389
The top 10 zones get 37% of the pickups


In [35]:
df_grouped = df2[['zone', 'pickups']]
df_grouped = df_grouped.groupby('zone').mean().sort_values('pickups', ascending=False).reset_index()

pickups_by_zone = alt.Chart(df_grouped).mark_bar().encode(
    x=alt.X('zone',
            sort=alt.Sort(field='pickups',
                         order='descending')
    ),
    y='pickups',
    tooltip='zone'
).properties(
    width=800
)
pickups_by_zone

alt.Chart(...)

# Map, pickups per zone
The following map shows a cloropeth map of zones by number of pickups

# Annotate everything

In [100]:
df3 = df[['LocationID', 'pickups']]
df3.shape

(586920, 2)

In [101]:
import geopandas as gpd

df_shape = gpd.read_file('../data/taxi_zones/taxi_zones.shp').to_crs(epsg=3785)
df_shape = df_shape.drop(['Shape_Area', 'Shape_Leng', 'OBJECTID'], axis=1)
df_shape = df_shape[df_shape['borough'] == 'Manhattan']

df3 = df[['LocationID', 'pickups']]
df3 = df3.groupby('LocationID').mean().reset_index()

df3 = df3.astype({"pickups": int})


df_shape_merged = df3.merge(df_shape, left_on='LocationID', right_on='LocationID')
df_shape_merged.head(200)

,LocationID,pickups,zone,borough,geometry
0,4,32,Alphabet City,Manhattan,"POLYGON ((-8234500.227 4971984.094, -8234502.1..."
1,12,6,Battery Park,Manhattan,"POLYGON ((-8239385.311 4968901.615, -8239356.3..."
2,13,121,Battery Park City,Manhattan,"POLYGON ((-8239027.255 4970990.635, -8239068.9..."
3,24,37,Bloomingdale,Manhattan,"POLYGON ((-8233137.952 4982697.872, -8233194.5..."
4,41,44,Central Harlem,Manhattan,"POLYGON ((-8231824.746 4984298.100, -8231526.5..."
5,42,20,Central Harlem North,Manhattan,"POLYGON ((-8230335.443 4988211.231, -8230345.5..."
6,43,176,Central Park,Manhattan,"POLYGON ((-8234586.991 4977725.736, -8234638.3..."
7,45,29,Chinatown,Manhattan,"POLYGON ((-8237364.516 4970257.968, -8237357.8..."
8,48,404,Clinton East,Manhattan,"POLYGON ((-8236660.189 4976319.581, -8236710.8..."
9,50,123,Clinton West,Manhattan,"POLYGON ((-8237272.410 4978991.629, -8237012.4..."


In [105]:
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LinearColorMapper, HoverTool
from bokeh.palettes import Cividis256 as palette
from bokeh.io import output_notebook, show
from shapely.geometry import Polygon, MultiPolygon
output_notebook()

def shape_data(shape_data):
    data = []
    for zonename, LocationID, shape, pickups in shape_data[["zone", "LocationID", "geometry", "pickups"]].values:
        #If shape is polygon, extract X and Y coordinates of boundary line:
        if isinstance(shape, Polygon):
            X, Y = shape.boundary.xy
            X = [int(x) for x in X]
            Y = [int(y) for y in Y]
            data.append([LocationID, zonename, X, Y, pickups])

        #If shape is Multipolygon, extract X and Y coordinates of each sub-Polygon:
        if isinstance(shape, MultiPolygon):
            for poly in shape:
                X, Y = poly.boundary.xy
                X = [int(x) for x in X]
                Y = [int(y) for y in Y]
                data.append([LocationID, zonename, X, Y, pickups])

    #Create new DataFrame with X an Y coordinates separated:
    shape_data = pd.DataFrame(data, columns=["LocationID", "ZoneName", "X", "Y", 'pickups'])
    return shape_data

shape_data = shape_data(df_shape_merged)

source = ColumnDataSource(shape_data)
max_passengers_per_hour = df_shape_merged['pickups'].max()
color_mapper = LinearColorMapper(palette=palette[::-1], high=max_passengers_per_hour, low=0)

p = figure(title="New York Taxi Pickups",
           plot_width=450, plot_height=750,
           toolbar_location=None,
           tools='pan,wheel_zoom,box_zoom,reset,save')

patches = p.patches(xs="X", ys="Y", source=source,
                  fill_color={'field': 'pickups', 'transform': color_mapper},
                  line_color="black")

hovertool = HoverTool(tooltips=[('ZoneName:', "@ZoneName"),
                                ("pickups:", "@pickups")])
p.add_tools(hovertool)
show(p)

Loading BokehJS ...

# Linear Chart, Pickups evolution over time

#### 4.1. Pickups evolution over Months
- The number of pickups over the year is quite constant.
- The only variation is a small decrease of pickups over the Summer months: July and August. This can be interpretate as that most of the taxis are taken by new yorkers (not tourists), and in this period they travel away from the city.

#### 4.2. Pickups evolution over Weeks
- This graph shows better how the taxi demand drops drastically on the USA Federal holidays:
    - New Year's Day: January.
    - Independance Day: 4th July.
    - Labor Day: first Monday of September.
    - Thanksgiving: 4th Thursday of November.
    
#### 4.3. Pickups evolution over a Single Week
- This graph shows the average number of pickups per weekday:
    - Monday and Sunday are a bit quieter than the other days, but not much.
    
#### 4.4. Pickups evolution over a Day
- This graph shows the average number of pickups over a day:
    - 18:00 and 19:00 are the peak hours.
    - 1:00am to 6:00am the quieter.

In [ ]:
# take 'datetime' and 'pickups' columns
df_grouped = df2[['datetime', 'pickups']]

# MONTHS DATA
df_grouped_months = df_grouped.groupby('datetime').mean().resample('M').mean()

# WEEKS DATA
df_grouped_weeks = df_grouped.groupby('datetime').mean().resample('W').mean()

# WEEKDAY DATA
df_grouped_weekday = df_grouped.groupby('datetime').mean().reset_index()
# create a categorical list to order weekday index
cats = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
# define key to groupby: weekday
key = df_grouped_weekday['datetime'].dt.weekday_name
# groupby 'key' (weekday) and reorder index using categorical values
df_grouped_weekday = df_grouped_weekday.groupby(key).mean().reindex(cats)

# HOURLY DATA
df_grouped_hour = df_grouped.set_index('datetime')
df_grouped_hour = df_grouped_hour.groupby(df_grouped_hour.index.map(lambda t: t.hour)).mean()

# PLOT MONTHS
df_grouped_months.plot(style="-o", figsize=(15,5))
plt.title('Pickups evolution over MONTHS')
plt.ylim(bottom=0)

# PLOT WEEKS
df_grouped_weeks.plot(style="-o", figsize=(15,5))
plt.title('Pickups evolution over WEEKS')
plt.ylim(bottom=0)

# PLOT WEEKDAYS
df_grouped_weekday.plot(style="-o", figsize=(15,5))
plt.title('Pickups evolution over A SINGLE WEEK')
plt.ylim(bottom=0)

# PLOT HOURS
df_grouped_hour.plot(style="-o", figsize=(15,5))
plt.title('Pickups evolution over A DAY')
plt.ylim(bottom=0)

plt.show()

# Scatter Plot, Relation between Precipitation and Pickups

Probar quitando outliers de precipitation.

Para ello dibujar un box plot de precipitatation

The scatter plot does not show a clear relation between 'precipitation' and 'pickups'.

The Histogram and Boxplot show that the mayority of the days it does not rain.

In [ ]:
df_grouped = df2[['datetime', 'pickups', 'precipitation']]
df_grouped = df_grouped.groupby('precipitation').mean().reset_index()

fig, ax =plt.subplots(1,3,figsize=(15,5))

sns.distplot(df2['precipitation'], ax=ax[0])
sns.boxplot(data=df2, y='precipitation', ax=ax[1])
sns.regplot(df_grouped['precipitation'], df_grouped['pickups'], ax=ax[2])

fig.show()

I will remove the days with zero precipitation and see again

In [ ]:
df2_prec = df2[df2['precipitation'] > 0]
df_grouped = df2_prec[['datetime', 'pickups', 'precipitation']]
df_grouped = df_grouped.groupby('precipitation').mean().reset_index()

fig, ax =plt.subplots(1,3,figsize=(15,5))

sns.distplot(df2_prec['precipitation'], ax=ax[0])
sns.boxplot(data=df2_prec, y='precipitation', ax=ax[1])
sns.regplot(df_grouped['precipitation'], df_grouped['pickups'], ax=ax[2])

fig.show()

I don´t still see any relationship. I will remove outliers above 0.5 precipitation.

In [ ]:
filter1 = df2['precipitation'] > 0
filter2 = df2['precipitation'] < 0.5
df2_prec = df2[filter1 & filter2]
df_grouped = df2_prec[['datetime', 'pickups', 'precipitation']]
df_grouped = df_grouped.groupby('precipitation').mean().reset_index()

fig, ax =plt.subplots(1,3,figsize=(15,5))

sns.regplot(df_grouped['precipitation'], df_grouped['pickups'], ax=ax[2])
sns.boxplot(data=df2_prec, y='precipitation', ax=ax[1])
sns.distplot(df2_prec['precipitation'], ax=ax[0])
fig.show()

#sns.regplot(df_grouped['precipitation'], df_grouped['pickups'])

The scatter plot is very clear: **there is no correlation between precipitation and pickups.**

# Pairwise Relationships
As the number of pickups is very stable over time, I will analyse only one month (so it runs faster in my computer)

In [ ]:
df2 = df[df['month'] == 2]
sns.pairplot(df2[['pickups','day', 'hour', 'LocationID',
       'dayofweek', 'isweekend', 'precipitation']])

Relations found:

- **Pickups - iswweekend**. There are more pickups during the weenkend.
- **Pickups - dayofweek**. There are more pickups on Saturday, Friday, Thursday. In this order. It is related to 'isweekend' but it contains more granularity about pickups distribution so I will keep this variable and remove 'is weekend'.
- **Pickups - hour**. There are more pickups between 23:00 and 3:00. This could be because the is not public transport.
- **Pickups - day**. There is a clear weekly pattern so this information is already given by 'dayofweek'. So I will remove 'day'.

# Pickups in Rainy Day vs Not Rainy Days

I have compared the average of pickups in rainy days vs not rainy days.

Against my prediction, there seems not to be relation at all between rain and pickups.

In [ ]:
df2 = df[['month', 'day', 'precipitation','pickups']]
df2 = df2.groupby(['month', 'day']).mean().reset_index()

# convert prediction variable y categorical. 0 = it didn´t rain. 1 = it rained.
df2.loc[(df2.precipitation != 0),'precipitation']=1
df2 = df2.groupby('precipitation')['pickups'].mean().reset_index()

df2.head(20)

# Some conclusions

- Remove features ```day``` and ```isweekend``` as they don´t add any information.
- There is not a strong correlation between ```precipitation``` and ```pickups```. However, I will train the models with and without ```precipitation``` to see if there is any improvement.
- Pickups are strongly affected by USA Fededal Holidays. It is important to create an ```isholiday``` feature.
- Number of pickups is very constant throught the year, with a small decrease of 15% in summer (July and August).
- Number of pickups is also constant throught the weeks, with visible decreases on the main USA Federal Holidays.
- The pickups evolution over a week is quite constant with small decreases on Sunday and Monday.
- Pickups evolution over a day is where I see the biggest variation, with 250+ pickups at 18:00-19:00pm and 50 between 1:00-6:00am.